# Exegesis

Preprocessing tool.


In [73]:
import re

In [74]:
FILE_NAME = "steiner"
INPUT_FILE = f"../../data/input/{FILE_NAME}.md"
OUTPUT_FILE = f"../../data/output/{FILE_NAME}.jsonl"

In [75]:
with open(INPUT_FILE, "r", encoding="utf-8") as f:
    lines = f.readlines()

In [76]:
def fix_ellipses(text):
    result = re.sub(r"(?<!\s)\.\.\.", "…", text)
    result = result.replace("...", "…")
    return result


def collapse_paragraph(lines: list[str]) -> list[str]:
    paragraphs = []
    current_paragraph = []

    for line in lines:
        if line.strip():
            current_paragraph.append(line.strip())
        elif current_paragraph:
            paragraphs.append(" ".join(current_paragraph))
            current_paragraph = []

    if current_paragraph:
        paragraphs.append(" ".join(current_paragraph))

    return paragraphs

In [77]:
# sub ... for …
# lines = [fix_ellipses(line) for line in lines]

# remove footnotes '(30)'
lines = [re.sub(r"\(\d+\)", "", line) for line in lines]

# drop titles
lines = [line for line in lines if not re.match("##", line)]

# collapse paragraphs
lines = collapse_paragraph(lines)

# removed multiple spaces
lines = [re.sub(r"\s+", " ", line) for line in lines]

In [78]:
with open(OUTPUT_FILE, "w", encoding="utf-8") as f:
    f.writelines(lines)

In [79]:
from typing import Generator


def make_jsonl(lines: list[str]) -> Generator:
    for i in range(len(lines)):

        if i == len(lines) - 1:
            break

        yield {
            "messages": [
                {"role": "user", "content": lines[i]},
                {"role": "assistant", "content": lines[i + 1]},
            ]
        }

In [ ]:
items = list(make_jsonl(lines))

In [80]:
import json

# save list. skip last item to not leave a \n hanging at the end
with open(OUTPUT_FILE, "w", encoding="utf-8") as f:
    for i, item in enumerate(items):
        json.dump(item, f)
        if i < len(items) - 1:
            f.write("\n")